# 9.8.2 Income Redistribution Model

Agent p's reward is, $u_p(s_p - x_p) = \frac{(s_p - x_p)^{1-\alpha_p}}{1-\alpha_p}$
,where $s_p$ is a predetermined level of wealth and $x_p$ is the amount of investment.

Wealth evolution is, $h_p(x_{p, t}, \epsilon_{p,t+1}) = \gamma_p x_{p,t} + \epsilon_{p,t+1} x_{p,t}^{\beta_p}$

Then the realized wealth for agent p at time t+1 is, $s_{p, t+1} = (1 - \psi) h_p(x_{p, t}, \epsilon_{p,t+1}) + \psi h_q(x_{q, t}, \epsilon_{q,t+1})$
,where $\psi$ is the pre-determined wealth share rate between the players.

Consider two players case, each denoted by 1 and 2, who live forever. 

The time independent state transition function is defined as follows.

$g(s_1, s_2, x_1, x_2, \epsilon_1, \epsilon_2) = ((1 - \psi) h_1(x_1, \epsilon_1)+ \psi h_2(x_2, \epsilon_2),\ (1 - \psi) h_2(x_2, \epsilon_2)+ \psi h_1(x_1, \epsilon_1))$

Using the above functions, and defining $V_p$ as the value functions, we can set the system of Bellman equations as follows.

$\begin{cases} V_1(s_1, s_2) = \max_{0\leq x_1\leq s_1} \left\{ u_1(s_1 - x_1) + \delta E_{\epsilon}\left[ V_1(g(s_1, s_2, x_1, x_2, \epsilon_1, \epsilon_2)) \right] \right\}\\[10pt]
V_2(s_1, s_2) = \max_{0\leq x_2\leq s_2} \left\{ u_2(s_2 - x_2) + \delta E_{\epsilon}\left[ V_2(g(s_1, s_2, x_1, x_2, \epsilon_1, \epsilon_2)) \right] \right\}
\end{cases}$

I solve the above systems of functional equations by using collocation method, which means searching the Markov perfect equilibrium in this model.

In [1]:
# alpha, beta, gamma, psi are model's parameters
mutable struct IRG
    alpha::Array{Float64,1}
    beta::Array{Float64,1}
    gamma::Array{Float64,1}
    psi::Float64
    delta::Float64
end

# functions in this model
# vector valued functions
function u(model::IRG, s::Array{Float64, 2}, x::Array{Float64, 2}, p::Int64)
    return ((s[:, p] - x[:, p]).^(1 - model.alpha[p]))./(1 - model.alpha[p])
end

function h(model::IRG, x::Array{Float64, 2}, epsilon::Float64, p::Int64)
    return model.gamma[p] * x[:, p] + epsilon * (x[:, p].^(model.beta[p]))
end

function s(model::IRG, x::Array{Float64, 2}, epsilon::Array{Float64})
    return [(1 - model.psi)*h(model, x, epsilon[1], 1) + model.psi*h(model, x, epsilon[2], 2), 
        (1 - model.psi)*h(model, x, epsilon[2], 2) + model.psi*h(model, x, epsilon[1], 1)]
end

s (generic function with 1 method)

Collocation method

In [2]:
using BasisMatrices

In [3]:
n = 5
smin = 3
smax = 11
maxit = 200;

In [5]:
# make collocation matrix
sgrid0 = linspace(smin, smax, n)
basis = Basis(SplineParams(sgrid0, 0, 3), SplineParams(sgrid0, 0, 3))
S, (coordx, coordy) = nodes(basis)
Φ = BasisMatrix(basis, Expanded(), S, 0)

BasisMatrix{BasisMatrices.Expanded} of order [0 0]

In [6]:
# make collocation function

# derivatives of  payoff, transition functions
# utility function
function ux(model::IRG, s::Array{Float64, 2}, x::Array{Float64, 2}, p::Int64)
    return -(s[:, p] - x[:, p]).^(-model.alpha[p])
end

function uxx(model::IRG, s::Array{Float64, 2}, x::Array{Float64, 2}, p::Int64)
    return -model.alpha[p]*(s[:, p] - x[:, p]).^(-model.alpha[p]-1)
end

# transition function
function sx(model::IRG, x::Array{Float64, 2}, epsilon::Array{Float64},  p::Int64)
    return (1 - model.psi)*(model.gamma[p] + epsilon[p]*model.beta[p]*x[:, p].^(model.beta[p] - 1))
end

function sxx(model::IRG, x::Array{Float64, 2}, epsilon::Array{Float64},  p::Int64)
    return (1 - model.psi)*epsilon[p]*model.beta[p]*(model.beta[p] - 1)*x[:, p].^(model.beta[p] - 1)
end

sxx (generic function with 1 method)

https://github.com/QuantEcon/CompEcon.jl/blob/master/src/core.jl

ここに書いてあるように、QuantEconのfunevalでorderを指定すると、中でbasisMatricesのorderを指定してsplineの微分による評価を得られるよ。

In [20]:
# Gaussian Quadrature
# using QuantEcon package for numerical integration w.r.t the expectation
# 3 nodes Gaussian quadrature of standard lognormal distribution
using QuantEcon
num_nodes = 3
nodes, weights = qnwlogn([num_nodes, num_nodes], [0,0], diagm([1, 1]))

# collocation function
function vmax(model::IRG, colnodes::Array{Float64, 2}, action::Array{Float64, 2}, coef::Array{Float64, 2})
    xnew = action
    for p in 1:2
        xl, xu = 0.0, colnodes[:, 1]
        for it in 1:maxit
            util, util_der1, util_der2 = u(model, colnodes, action, p), ux(model, colnodes, action, p), uxx(model, colnodes, action, p) 
            Ev, Evx, Evxx = 0.0, 0.0, 0.0
            for k in 1:num_nodes^2
                eps, weight= nodes[k, :], weights[k]
                transition, transition_der1, transition_der2 = s(model, action, eps, p), sx(model, action, eps, p), sxx(model, action, eps, p)
                # evaluation based on the basis functions
                vn, vnder1, vnder2 = funeval(coef[:, p], basis, transition), funeval(coef[:, p], basis, transition, 1), funeval(coef[:, p], basis, transition, 2)
                Ev += weight * vn
                Evx += weight* vnder1.* transition_der1
                Evxx += vnder1.*transition_der2 + vnder2 .* (transition_der1.^2)
            end
            v = util + Ev
            # improvement by Newton method
            delx = -(util_der1 + delta * Evx) ./ (util_der2 + delta*Evxx)
            delx = min(max(delx, xl-x), xu-x)
            x[:, p] = x[:, p] + delx
            if abs(delx) < tol
                break
            end
        end
        xnew[:, p] = x[:, p]
    end
    return v, xnew
end

initial = zeros(size(Φ.vals[1])[1])
x = zeros(size(Φ.vals[1])[1])
model = IRG([0.2,0.2], [0.5, 0.5], [0.9,0.9], [0.1,0.1], 0.05, 0.9)
tol = 0.01
# iteration for coefficients
c = initial
for it in 1:maxit
    cold = c
    v, x = vmax(model, S, x, c)
    c = Φ.vals[1] \ v
    if norm(cold - c) < tol
        break
    end
end

LoadError: [91mUndefVarError: model not defined[39m

In [17]:
size(Φ.vals[1])

(49, 49)

In [18]:
zeros(2)

2-element Array{Float64,1}:
 0.0
 0.0